In [1]:
!pip install lightning -q

^C
ERROR: Operation cancelled by user


In [71]:
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import functools
from transformers import AdamW, get_linear_schedule_with_warmup
import math
from time import time
from datasets import load_dataset, Dataset
import transformers

In [73]:
import transformers
import torch
import datasets
print(transformers.__version__)
print(torch.__version__)
print(datasets.__version__)

4.30.2
2.0.1
2.13.1


In [3]:
path_train = "/Users/datle/Downloads/ELI5.jsonl"
path_val = "/Users/datle/Downloads/ELI5_val.jsonl"
dataset_train = load_dataset('json', data_files = path_train)
dataset_val = load_dataset('json', data_files = path_val)
train = dataset_train['train'].select(range(1,100))
val = dataset_val['train'].select(range(1,100))

Found cached dataset json (/Users/datle/.cache/huggingface/datasets/json/default-0913b0ed92067c10/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (/Users/datle/.cache/huggingface/datasets/json/default-f1b40a908fd17224/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

In [5]:
def replace_text(string):
    return string.replace("\'","").replace("\n","").replace("URL_0","").lower().strip()
def concat(ex, n):
    question = replace_text(ex['question'])
    context = ex['ctxs'][:n]
    if type((context[0])) == list:
        context = [k[0] for k in context]
    context = replace_text(' '.join(context))
    ex['ques_ctxs'] = f"question: {question} context: {context}"
    return ex

In [6]:
question_ds_train = train.map(lambda x: concat(x, n=2), remove_columns = ['question', 'ctxs'])
question_ds_val = val.map(lambda x: concat(x, n=2), remove_columns = ['question', 'ctxs'])

Loading cached processed dataset at /Users/datle/.cache/huggingface/datasets/json/default-0913b0ed92067c10/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-18d0e246eb96737f.arrow
Loading cached processed dataset at /Users/datle/.cache/huggingface/datasets/json/default-f1b40a908fd17224/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-367add5173f00b31.arrow


In [8]:
def tokenize(ex):
    dct = tokenizer(ex['ques_ctxs'],max_length=128, 
                    padding='max_length', truncation=True, return_tensors='pt')
    ex['input_id_q'], ex['attention_q'] = dct['input_ids'][0], dct['attention_mask'][0]
    
    dct1 = (tokenizer(ex['answers'],max_length=128, 
                    padding='max_length', truncation=True, return_tensors='pt'))
    ex['input_id_a'], ex['attention_a']= dct1['input_ids'], dct1['attention_mask']
    
    return ex

In [9]:
tokenized_ds_train = question_ds_train.map(tokenize, remove_columns = ['question_id','answers','ques_ctxs'])
tokenized_ds_val = question_ds_val.map(tokenize, remove_columns = ['question_id','answers','ques_ctxs'])

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1507 [00:00<?, ? examples/s]

In [10]:
tokenized_ds_train

Dataset({
    features: ['input_id_q', 'attention_q', 'input_id_a', 'attention_a'],
    num_rows: 999
})

In [58]:
class eli5dataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.qa_id_list = [
            (i,j)
            for i, q in enumerate(self.dataset['input_id_a'])
            for j, a in enumerate(q)
        ]
    def __len__(self):
        return len(self.qa_id_list)
    def make_example(self, idx):
        q_ids = []
        q_mask = []
        a_ids = []
        a_mask = []
        if type(idx) == int:
            idx=[idx]
        for m in idx:
            i,j = self.qa_id_list[m]
            
        
            q_ids.append(self.dataset['input_id_q'][i])
            q_mask.append(self.dataset['attention_q'][i])
            a_id = self.dataset['input_id_a'][i][j]
            a_m = self.dataset['attention_a'][i][j]
            a_ids.append(a_id)
            a_mask.append(a_m)
        
        q_ids, q_mask = (
        torch.squeeze(torch.LongTensor(q_ids)),
        torch.squeeze(torch.LongTensor(q_mask))
        )
        print('q_ids shape',q_ids.shape)
        print('q_mask shape', q_mask.shape)
        a_ids, a_mask = (
        torch.LongTensor(a_ids),
        torch.LongTensor(a_mask)
        )
        
        print('a_ids shape', a_ids.shape)
        print('a_mask shape', a_mask.shape)
        labels = a_ids[:, 1:].contiguous().clone()
        labels[a_mask[:, 1:].contiguous() == 0] = -100
        print("labels shape", labels.shape)
        model_inputs = {
        'input_ids': q_ids,
        'attention_mask': q_mask,
        'decoder_input_ids': a_ids[:, :-1].contiguous(),
        'labels': labels,
    }
        return model_inputs
    def __getitem__(self, idx):
        print(idx)
        return self.make_example(idx)

In [60]:
def data_loader(dataset, args):
    train_sampler = SequentialSampler(dataset)
#     model_collate_fn = functools.partial(make_qa_s2s_batch,)
    data_loader = DataLoader(dataset, batch_size=args.batch_size, sampler=train_sampler)
    return data_loader

In [61]:
class ArgumentsS2S():
    def __init__(self):
        self.batch_size = 2
        self.max_length = 128

s2s_args = ArgumentsS2S()

In [62]:
import lightning as L
class bart_model(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
    def forward(self, batch_input):
        output = self.model(**batch_input)
        return output.loss, output.logits
    def training_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('train_loss',loss, prog_bar=True, logger=True)
        return loss
    def validation_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('val_loss',loss, prog_bar=True, logger=True)
        return loss
    def test_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('test_loss',loss, prog_bar=True, logger=True)
        return loss
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=2e-4)

In [63]:
train1 = eli5dataset(tokenized_ds_train)
val1 = eli5dataset(tokenized_ds_val)
train_1 = data_loader(train1, s2s_args)
val_1 = data_loader(val1, s2s_args)


In [64]:
next(iter(train_1))

[0, 1]
q_ids shape torch.Size([2, 128])
q_mask shape torch.Size([2, 128])
a_ids shape torch.Size([2, 128])
a_mask shape torch.Size([2, 128])
labels shape torch.Size([2, 127])


{'input_ids': tensor([[    0, 40018,    35,   596,    32,   430, 34325,    36, 32786, 28696,
           1084, 28696,  4549,    43,     9,  1123,   850,   818,   460,   158,
           3205,   430,   116,  5377,    35,    33, 10341,    57,   130,     7,
            237,   498,     5,   425,    11,     5, 10409,   982,     6,    19,
            850,   148,  3788,  2383, 32857,     9,  4480,   134,     4,   612,
             73, 13534,   241,    36,  9006,    68,   134,     4,  3714,   228,
           6474,   241,  1358,   245,     4,   306,    73,   687, 50141,  9487,
             43,   150,     5,   201,    56,   850,   198,    68,   134,     4,
           1096,   228,   201, 17126,  1358,   288,     4,  1749,    73,   462,
            322, 10669,    10,   739,   712,   454,     5,  1035,     9,  2266,
              6,     5,   253,     9,  2266,  2984,    10,   670,  2991,  3307,
             19,    10,  4406,   776, 14013,     6,    19,     5,   674,   425,
              9,  2423,    

In [65]:
my_model = bart_model()

In [66]:
trainer = L.Trainer(max_epochs=3)
#accelerator='gpu', devices=2
trainer.fit(my_model, train_1, val_1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
-------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/datle/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


[0, 1]
q_ids shape torch.Size([2, 128])
q_mask shape torch.Size([2, 128])
a_ids shape torch.Size([2, 128])
a_mask shape torch.Size([2, 128])
labels shape torch.Size([2, 127])
[2, 3]
q_ids shape torch.Size([2, 128])
q_mask shape torch.Size([2, 128])
a_ids shape torch.Size([2, 128])
a_mask shape torch.Size([2, 128])
labels shape torch.Size([2, 127])


/Users/datle/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

[0, 1]
q_ids shape torch.Size([2, 128])
q_mask shape torch.Size([2, 128])
a_ids shape torch.Size([2, 128])
a_mask shape torch.Size([2, 128])
labels shape torch.Size([2, 127])
[2, 3]
q_ids shape torch.Size([2, 128])
q_mask shape torch.Size([2, 128])
a_ids shape torch.Size([2, 128])
a_mask shape torch.Size([2, 128])
labels shape torch.Size([2, 127])
[4, 5]
q_ids shape torch.Size([2, 128])
q_mask shape torch.Size([2, 128])
a_ids shape torch.Size([2, 128])
a_mask shape torch.Size([2, 128])
labels shape torch.Size([2, 127])
[6, 7]
q_ids shape torch.Size([2, 128])
q_mask shape torch.Size([2, 128])
a_ids shape torch.Size([2, 128])
a_mask shape torch.Size([2, 128])
labels shape torch.Size([2, 127])
[8, 9]


/Users/datle/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
